In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

In [7]:
fileName = 'GSR_hrac1.xlsx'
dataHra2 = pandas.read_excel(fileName, sheet_name="Hra_2")
dataCopy = dataHra2

In [8]:
# přepis stresové hodnoty 2 na 1
dataCopy.loc[(dataCopy.Stres == 2), "Stres"] = 1
impedance = list(dataCopy["Data"].copy().to_numpy())
stres = list(dataCopy["Stres"].copy().to_numpy())

In [9]:
def rozdelitStres(data, stav):
    """
    Funkce rozdělí sloupec hodnot impedancí do skupin podle stresové stavu.
    Výstupem je seznam skupin impedančních hodnot.
    """

    rozdelanaImpedance = []
    currentValues = []

    for index in range(len(data)):
        if index == len(data) - 1: # ukončení cyklu, pokud je index na posledním řádku
            rozdelanaImpedance.append(currentValues)
            break
        
        # zápis hodnot impedance v úseku se stejným stresovým stavem
        if stav[index] == stav[index + 1]:
            currentValues.append(data[index])
        # zápis skupiny hodnot při změně stresového stavu
        elif stav[index] != stav[index + 1]:
            currentValues.append(data[index])
            rozdelanaImpedance.append(currentValues)
            currentValues = []

    klidovyStav = rozdelanaImpedance[0::2]
    stresovyStav = rozdelanaImpedance[1::2]

    return klidovyStav, stresovyStav

In [17]:
skupinaKlid, skupinaStres = rozdelitStres(impedance, stres)
len(skupinaKlid), len(skupinaStres)

(25, 24)

In [24]:
def prumerSkupin(seznamSkupin):
    """
    Funkce vypočítá průměrnou hodnotu impedance pro každou skupinu.
    """
    
    seznamPrumeru = []
    for skupina in seznamSkupin:
        seznamPrumeru.append(np.mean(skupina))
    return seznamPrumeru

def smerOdchylkaSkupin(seznamSkupin):
    """
    Funkce vypočítá směrodatnou odchylku impedance pro každou skupinu.
    """

    seznamSmerOdchylek = []
    for skupina in seznamSkupin:
        seznamSmerOdchylek.append(np.std(skupina))
    return seznamSmerOdchylek

In [25]:
klidPrumer = prumerSkupin(skupinaKlid)
klidSmerOdchylka = smerOdchylkaSkupin(skupinaKlid)
stresPrumer = prumerSkupin(skupinaStres)
stresSmerOdchylka = smerOdchylkaSkupin(skupinaStres)